In [1]:
%showtypes on 

Types will be printed.
Types will be printed.


In [2]:
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx._
import scala.util.Random

# Create link graph

In [3]:
val arr = List.fill(30)(
    (Random.nextLong.abs % 10, Random.nextLong.abs % 10, Random.nextLong.abs % 3)
)

val rdd: RDD[Edge[Long]] = sc.parallelize(arr).map(e => Edge(e._1, e._2, e._3))

arr: List[(Long, Long, Long)] = List((7,9,1), (0,5,1), (9,6,1), (8,3,2), (2,4,1), (2,0,1), (1,4,1), (9,1,1), (7,0,1), (0,2,0), (2,7,1), (8,1,0), (9,0,2), (2,6,1), (5,2,1), (9,4,0), (8,3,2), (4,7,2), (9,4,0), (1,5,0), (9,3,2), (3,5,0), (8,9,0), (1,2,2), (8,9,0), (7,3,0), (8,8,0), (5,9,0), (3,9,0), (6,3,0))
rdd: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Long]] = MapPartitionsRDD[1] at map at <console>:33


rdd: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Long]] = MapPartitionsRDD[1] at map at <console>:33


In [4]:
val gr: Graph[Double, Long] = Graph.fromEdges(rdd, 1.0)

gr.triplets.take(3).foreach(println)

((7,1.0),(9,1.0),1)
((0,1.0),(5,1.0),1)
((9,1.0),(6,1.0),1)


gr: org.apache.spark.graphx.Graph[Double,Long] = org.apache.spark.graphx.impl.GraphImpl@37aa270f


gr: org.apache.spark.graphx.Graph[Double,Long] = org.apache.spark.graphx.impl.GraphImpl@37aa270f


In [5]:
val graph = Graph(gr.vertices, gr.edges ++ gr.edges.reverse).
    mapVertices{ (vid, attr) => (vid, Set[Long]()) }

graph.vertices.take(3).foreach(println)

(0,(0,Set()))
(1,(1,Set()))
(2,(2,Set()))


graph: org.apache.spark.graphx.Graph[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[Long]),Long] = org.apache.spark.graphx.impl.GraphImpl@6f60c65b


graph: org.apache.spark.graphx.Graph[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[Long]),Long] = org.apache.spark.graphx.impl.GraphImpl@6f60c65b


# Define vprog, sendMsg, sendMsg

In [6]:
type M = (Long, Set[Long])
type A = (Long, Set[Long])

val initMsg: M = (9L, Set[Long]())

defined type alias M
defined type alias A
initMsg: M = (9,Set())


initMsg: M = (9,Set())


In [7]:
def vprog(vid: VertexId, attr: A, inMsg: M): A = 
    ( attr._1 min inMsg._1, attr._2 ++ inMsg._2)

def sendMsg(e: EdgeTriplet[A, Long]): Iterator[(VertexId, M)] = {
    if (e.srcAttr._1 == e.dstAttr._1 && e.dstAttr._2.contains(e.attr) ) Iterator.empty
    else Iterator((e.dstId, (e.srcAttr._1, Set(e.attr) ++ e.srcAttr._2 )))
}

// def sendMsg(e: EdgeTriplet[A, Long]): Iterator[(VertexId, M)] = {
//     Iterator((e.dstId, (e.srcAttr._1, Set(e.attr) ++ e.srcAttr._2)))
// }

def mergeMsg(a: M, b: M): M = 
    (a._1 min b._1, a._2 ++ b._2)

vprog: (vid: org.apache.spark.graphx.VertexId, attr: A, inMsg: M)A
sendMsg: (e: org.apache.spark.graphx.EdgeTriplet[A,Long])Iterator[(org.apache.spark.graphx.VertexId, M)]
mergeMsg: (a: M, b: M)M


# Pregel

In [8]:
graph.
    pregel(initMsg, 10)(vprog, sendMsg, mergeMsg).
    vertices.
    take(3).
    foreach(println)

(0,(0,Set(1, 0, 2)))
(1,(0,Set(1, 0, 2)))
(2,(0,Set(1, 0, 2)))
